In [1]:
# %load_ext autoreload
# %autoreload 2
# turn off autoreload so that we can use the old model 
# when editing the current project

from coeditor.common import *
import os

os.chdir(proj_root())

In [2]:
from coeditor.retrieval_model import RetrievalEditorModel
from coeditor.api import EditPredictionService, BasicQueryEditEncoder, BatchArgs

model_path = get_model_dir(False) / "coeditor-large-query-stub/checkpoint-248892"
model = RetrievalEditorModel.load(model_path)
model.to("cuda:1")
batch_args = BatchArgs.service_default()
service = EditPredictionService(
    proj_root(),
    model,
    batch_args=batch_args,
    encoder=BasicQueryEditEncoder(
        max_ref_tks=batch_args.max_ref_tks,
        max_query_tks=batch_args.max_query_tks,
        max_output_tks=batch_args.max_output_tks,
    ),
)


/home/jiayi/Projects/SPOT/src/coeditor/retrieval_model.py:303: UserWarning: No extra args found, using default model setting.
  warnings.warn("No extra args found, using default model setting.")


In [5]:
service.suggest_edit(Path("src/coeditor/encoders.py"), 146)
# service.suggest_edit(Path("src/coeditor/api.py"), 277)
# service.suggest_edit(Path("src/spot/utils.py"), (218, 0))


stub files: dict_keys(['coeditor.api', 'coeditor.encoders', 'spot.static_analysis'])
num batches: 1
Batch stats: {'mean': '1.0', 'median': '1.0', 'min': '1.0', 'max': '1.0'}
========== Predicted code change ==========
* Modified: 
    ... 3 lines omitted ...
        class BasicQueryEditEncoder(EditEncoder[BasicTkQueryEdit]):
            def encode_pedit(
                self,
                pedit: ProjectEdit,
                stub_cache: TimedCache[ModuleName, list[TokenSeq], int],
                include_additions: bool = False,
    -           queries: Iterable[Change[PythonFunction]] | None = None,
    +           queries: Sequence[Change[PythonFunction]] | None = None,
            ) -> Iterable[BasicTkQueryEdit]:
                ctx_enc = CtxEncoder(pedit, self.collapse_unchanged)
    -           moved = find_moved(pedit.all_elem_changes())
    -           moved_paths = {a for a, b in moved} | {b for a, b in moved}
    +           renamed = find_renamed(pedit.all_elem_changes())
 

In [4]:
service.tlogger.as_dataframe()


,name,count,avg_time,total_time
1,construct project edit,1,3.296461,3.296461
3,run model,1,1.625003,1.625003
2,encode edits,1,0.676539,0.676539
0,get target element,1,0.158115,0.158115
